# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

## Задание 1. 

### Обязательная часть

Будем парсить страницу со свежеми новостям на [habr.com/ru/all/](https://habr.com/ru/all/).

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

`KEYWORDS = ['python', 'парсинг']`

 Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы). 
 
В итоге должен формироваться датафрейм вида: `<дата> - <заголовок> - <ссылка>`

### Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.  

Итоговый датафрейм формировать со столбцами: `<дата> - <заголовок> - <ссылка> - <текст_статьи>`


In [1]:
import pandas as pd

In [2]:
import requests

In [3]:
import time

In [4]:
from bs4 import BeautifulSoup

In [5]:
URL1 = 'https://habr.com/ru/all/'
KEYWORDS = ['python', 'парсинг']
PAGE = 5

In [6]:
def post_links():
    b_list = []
    time.sleep(0.2)
    for i in range(1,PAGE+1):
        soup = BeautifulSoup(requests.get(URL1+'page'+str(i)).text, 'html.parser')
        links = soup.find_all('h2', class_='post__title')
        a_list = list(map(lambda x: x.find('a').get('href'), links))
        for x in a_list:
            b_list.append(x) #сюда можно добавить if чтобы например оставить только посты или только блоги
    return b_list

In [7]:
%%time

post_links()

Wall time: 3.38 s


['https://habr.com/ru/company/itsoft/blog/556038/',
 'https://habr.com/ru/post/556028/',
 'https://habr.com/ru/post/556024/',
 'https://habr.com/ru/post/556018/',
 'https://habr.com/ru/company/piter/blog/555860/',
 'https://habr.com/ru/company/it_people/blog/555916/',
 'https://habr.com/ru/company/rshb/blog/556004/',
 'https://habr.com/ru/company/skillfactory/blog/555914/',
 'https://habr.com/ru/post/556000/',
 'https://habr.com/ru/company/itsoft/blog/555998/',
 'https://habr.com/ru/post/555988/',
 'https://habr.com/ru/post/555934/',
 'https://habr.com/ru/post/555912/',
 'https://habr.com/ru/post/555982/',
 'https://habr.com/ru/company/otus/blog/555980/',
 'https://habr.com/ru/post/555976/',
 'https://habr.com/ru/company/skillfactory/blog/555770/',
 'https://habr.com/ru/post/555910/',
 'https://habr.com/ru/post/555954/',
 'https://habr.com/ru/post/555938/',
 'https://habr.com/ru/company/geekbrains/blog/555932/',
 'https://habr.com/ru/company/otus/blog/555924/',
 'https://habr.com/ru/po

In [8]:
def post_find():
    habr_news = pd.DataFrame()
    for link in post_links():
        soup = BeautifulSoup(requests.get(link).text, 'html.parser')
        prewiew = soup.find('span', class_='post__title-text').text.lower()
        post = soup.find('div', class_='post__body post__body_full').text.lower()
        time.sleep(0.2)
        for x in KEYWORDS:
            if prewiew.find(x) > 0:
                date = pd.to_datetime(soup.find('span', class_='post__time').get('data-time_published'), dayfirst=True).date()
                row = {'date': date, 'title': prewiew, 'link': link, 'text': post}
                habr_news = pd.concat([habr_news, pd.DataFrame([row])])
            elif post.find(x) > 0:
                date = pd.to_datetime(soup.find('span', class_='post__time').get('data-time_published'), dayfirst=True).date()
                row = {'date': date, 'title': prewiew, 'link': link, 'text': post}
                habr_news = pd.concat([habr_news, pd.DataFrame([row])])  
    return habr_news.reset_index().drop_duplicates()[['date','title','link','text']] #если в теле статьи попадаются оба искомых варианта, возникают дубли

In [9]:
%%time

post_find()

Wall time: 1min 4s


,date,title,link,text
0,2021-05-05,как я посчитал миллионное число фибоначчи,https://habr.com/ru/company/skillfactory/blog/...,"\nвсе мы понимаем, что рекурсивное вычисление ..."
1,2021-05-05,"python, наука о данных и выборы: часть 1",https://habr.com/ru/post/556000/,\nсерия из 5 постов для начинающих представляе...
2,2021-05-05,использование пейджера в xxi веке,https://habr.com/ru/company/skillfactory/blog/...,\nпейджеры были очень популярны в 90-е годы. в...
3,2021-05-05,врач со знанием data science – путь к уникальн...,https://habr.com/ru/company/geekbrains/blog/55...,\nкак сближение медицины и data science открыв...
4,2021-05-05,моржовый оператор в python,https://habr.com/ru/company/otus/blog/555924/,"\nперевод подготовлен в рамках курса ""python d..."
5,2021-05-05,пример архитектуры аналитического решения с ис...,https://habr.com/ru/company/epam_systems/blog/...,"\nвсем привет, меня зовут максим крупенин, я р..."
6,2021-05-05,тонкое искусство быть девелопер адвокатом,https://habr.com/ru/company/haulmont/blog/555902/,\n\nот переводчика: профессия девелопер адвока...
7,2021-05-05,что недоговаривают тинькофф инвестиции. вытаск...,https://habr.com/ru/post/555884/,\nтинькофф инвестиции - популярный российский ...
8,2021-05-05,как получить доступ из одного докер-контейнера...,https://habr.com/ru/post/554190/,\nизображение от mike wheatleyцель этой статьи...
9,2021-05-05,fuchsia os от google выходит из тени: ее устан...,https://habr.com/ru/company/selectel/blog/554820/,\n\r\nна хабре многие внимательно следят за су...


## Задание 2.

### Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса [Avast Hack Ckeck](https://www.avast.com/hackcheck/).
Список email-ов задаем переменной в начале кода:  
`EMAIL = [xxx@x.ru, yyy@y.com]`

В итоге должен формироваться датафрейм со столбцами: `<дата утечки> - <источник утечки> - <описание утечки>`  

**Подсказка**: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [10]:
URL2 = 'https://www.avast.com/hackcheck/'
URL3 = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
EMAIL = ['xxx@x.ru', 'yyy@y.com']

In [11]:
HEADERS = {
    'Vaar-Version': '0',
    'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
    'Vaar-Header-App-Build-Version': '1.0.0',
}

In [12]:
import json

In [13]:
def email_check ():
    pd1 = None
    for x in EMAIL:
        y = [x]
        res = json.loads(requests.post(URL3,json = {'emailAddresses':y}, headers = HEADERS).content) #json.load иначе тип данных bytes
        if res is None:
            return print('Опять эта блокировка, о Боги!') #это просто кошмар
        pd2 = pd.DataFrame.from_dict(res['breaches']).transpose()
        pd2['email'] = x
        if pd1 is None:
            pd1 = pd2
        else:
            pd1 = pd.concat([pd1, pd2]).reset_index()
    return pd1[['email','publishDate','site','description']]

email_check ()

,email,publishDate,site,description
0,xxx@x.ru,2019-03-28T00:00:00Z,verifications.io,Big data e-mail verification platform verifica...
1,xxx@x.ru,2020-05-21T00:00:00Z,vk.com,"At some time in 2020, the Russian social netwo..."
2,xxx@x.ru,2017-02-14T00:00:00Z,parapa.mail.ru,"In July and August 2016, two criminals execute..."
3,xxx@x.ru,2016-10-29T00:00:00Z,vk.com,Popular Russian social networking platform VKo...
4,xxx@x.ru,2016-10-21T00:00:00Z,adobe.com,"In October of 2013, criminals penetrated Adobe..."
5,xxx@x.ru,2017-02-14T00:00:00Z,cfire.mail.ru,"In July and August of 2016, two criminals carr..."
6,xxx@x.ru,2017-01-31T00:00:00Z,cdprojektred.com,"In March 2016, CDProjektRed.com.com's forum da..."
7,xxx@x.ru,2016-10-23T00:00:00Z,imesh.com,"In June 2016, a cache of over 51 million user ..."
8,yyy@y.com,2019-03-28T00:00:00Z,verifications.io,Big data e-mail verification platform verifica...
9,yyy@y.com,2019-02-21T00:00:00Z,www.dangdang.com,"This is a list of email addresses only, and as..."


### Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.  
Документация к API VK: https://vk.com/dev/methods
, вам поможет метод [wall.get](https://vk.com/dev/wall.get)  
```
GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ  
```

В итоге должен формироваться датафрейм со столбцами: `<дата поста> - <текст поста>`